In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\arbas\\Documents\\Projects\\Machine-Learning\\EndToEnd_Deployed\\Brain_Tumor_Classification'

In [7]:
import os
import sys
from src.logger import logging
from src.utils import MainUtils

In [8]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class Data_Ingestion_Config:
    root_dir: Path
    source_URL: str
    local_data_path : Path 
    unzip_dir: Path

In [19]:
from src.constant import *
# from src.utils.common import read_yaml,create_directories

In [ ]:
def create_directories(file_path):
    pass
def read_yaml(file_path):
    pass
class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config = read_yaml(config_path)
        self.config = read_yaml(config_path)
        self.config = read_yaml(config_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->Data_Ingestion_Config:
        config = self.config.Data_Ingestion
        create_directories([config.root_dir])
        data_ingestion_config = Data_Ingestion_Config(root_dir=config.root_dir,source_URL=config.source_URL,local_data_path=config.loca_data_path,unzip_dir=config.unzip_dir)
        return data_ingestion_config
        

In [ ]:
import os
import urllib.request as request
import zipfile
from src.Project4.utils.common import get_size, logger


class Data_Ingestion:
    def __init__(self,config:Data_Ingestion_Config):
        self.config = config
    
    def initiate_data_ingestion(self):
        train_dir = os.path.join('dataset/Training/')
        test_dir = os.path.join('dataset/Testing/')
        self.utils.copy_images(
            source_folder=train_dir, destination_folder=self.data_ingestion_config.train_path)
        self.utils.copy_images(
            source_folder=test_dir, destination_folder=self.data_ingestion_config.test_path)

In [ ]:
try:
    config = Configuration_Manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Data_Ingestion(data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    raise e